In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={

    'Vendedor':

    """
    
    SELECT [ID Vendedor],[Nome Resumido],[ID Equipe],DDD,Telefone 
    FROM netfeira.vw_vendedor
    WHERE [Status do Vendedor]='ATIVO' AND Categoria='CLT' AND Telefone IS NOT NULL
    
    """,

    'Supervisor':

    """
    
    SELECT [ID Equipe],Equipe,[ID Sup],Supervisor,[DDD Sup],[Telefone Sup],
    [ID Gerente],Gerente,[DDD Gerente],[Telefone Gerente]
    FROM netfeira.vw_supervisor
    WHERE NOT Equipe LIKE '%120%'
    
    """,

    'Aberto':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT Pedido,[ID Cliente],[ID Vendedor],SKU,[Unid. VDA],[Qtde VDA],[Valor VDA],[Total Venda]
	FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=@DTBASE
    
    """,
    
    'Cliente':
    
    """
    
    SELECT [ID Cliente],[Razão Social],[Nome Fantasia]
    FROM netfeira.vw_cliente    
    
    """,
    
    'Produto':
    
    """
    
    SELECT SKU,Produto,Fabricante
    FROM netfeira.vw_produto    
    
    """

}

In [2]:
df=sql.CriarTabela(kwargs=querys)

In [3]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Nome Resumido', 'ID Equipe', 'DDD', 'Telefone'], dtype='object')

In [4]:
df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [5]:
df['Aberto'].columns

Index(['Pedido', 'ID Cliente', 'ID Vendedor', 'SKU', 'Unid. VDA', 'Qtde VDA',
       'Valor VDA', 'Total Venda'],
      dtype='object')

In [6]:
df['Aberto']

,Pedido,ID Cliente,ID Vendedor,SKU,Unid. VDA,Qtde VDA,Valor VDA,Total Venda
0,443618,10075,JOSENUNE,9447,CX,2.0,115.00,230.00
1,443618,10075,JOSENUNE,9448,CX,2.0,115.00,230.00
2,443618,10075,JOSENUNE,9445,CX,2.0,115.00,230.00
3,443618,10075,JOSENUNE,9446,CX,2.0,115.00,230.00
4,443619,23732,JOSENUNE,9447,CX,1.0,115.00,115.00
...,...,...,...,...,...,...,...,...
573,443803,15917,CLAUDIOP,9596,CX,1.0,232.80,232.80
574,443803,15917,CLAUDIOP,1044,KG,1.0,21.05,21.05
575,443803,15917,CLAUDIOP,666,KG,1.0,19.52,19.52
576,443803,15917,CLAUDIOP,139,KG,1.0,49.03,49.03


In [7]:
df['Analise']=df['Aberto'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Analise']['Pedidos']=df['Analise']['ID Vendedor'].apply(lambda codigo: len(df['Aberto']['Pedido'].loc[(df['Aberto']['ID Vendedor']==codigo)].unique().tolist()))

df['Analise']['Clientes']=df['Analise']['ID Vendedor'].apply(lambda codigo: len(df['Aberto']['ID Cliente'].loc[(df['Aberto']['ID Vendedor']==codigo)].unique().tolist()))

df['Analise']['Produtos']=df['Analise']['ID Vendedor'].apply(lambda codigo: len(df['Aberto']['SKU'].loc[(df['Aberto']['ID Vendedor']==codigo)].unique().tolist()))

df['Analise']

,ID Vendedor,Total Venda,Pedidos,Clientes,Produtos
0,ADANTAS,3968.30,3,3,4
1,ADELIALO,2778.79,5,5,11
2,ALERUIZ,2602.02,2,2,5
3,AVULSO,189.28,4,2,6
4,CLAUDIOP,4384.09,4,4,14
5,CRIS,784.06,1,1,3
6,CRISTIAN,617.70,2,2,7
7,DINHO,869.48,1,1,2
8,DOUGLASM,7816.90,14,14,13
9,ELENICEV,4418.11,2,2,12


In [8]:
df['Vendedor']=df['Vendedor'].merge(df['Supervisor'],on='ID Equipe',how='inner')

df['Vendedor']=df['Vendedor'][['ID Vendedor', 'Nome Resumido', 'DDD', 'Telefone',
       'Equipe']]

df['Vendedor']=df['Vendedor'].merge(df['Analise'],on='ID Vendedor',how='inner')

df['Vendedor'].rename(columns={'Total Venda':'Realizado R$'},inplace=True)

In [9]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Nome Resumido', 'DDD', 'Telefone', 'Equipe',
       'Realizado R$', 'Pedidos', 'Clientes', 'Produtos'],
      dtype='object')

In [10]:
df['Aberto']=df['Aberto'].merge(df['Vendedor'],on='ID Vendedor',how='inner')[['Pedido', 'ID Cliente', 'ID Vendedor','Nome Resumido', 'Equipe', 'SKU', 'Unid. VDA', 'Qtde VDA',
       'Valor VDA', 'Total Venda']]

In [11]:
df['Aberto'].columns

Index(['Pedido', 'ID Cliente', 'ID Vendedor', 'Nome Resumido', 'Equipe', 'SKU',
       'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda'],
      dtype='object')

In [12]:
df['Cliente'].columns

Index(['ID Cliente', 'Razão Social', 'Nome Fantasia'], dtype='object')

In [13]:
df['Produto'].columns

Index(['SKU', 'Produto', 'Fabricante'], dtype='object')

In [14]:
df['Aberto']=df['Aberto'].merge(df['Cliente'],on='ID Cliente',how='inner')[['Pedido', 'ID Cliente','Razão Social', 'Nome Fantasia', 'ID Vendedor','Nome Resumido', 'Equipe', 'SKU', 'Unid. VDA', 'Qtde VDA',
       'Valor VDA', 'Total Venda']]

In [15]:
df['Aberto']=df['Aberto'].merge(df['Produto'],on='SKU',how='inner')[['Pedido', 'ID Cliente','Razão Social', 'Nome Fantasia', 'ID Vendedor','Nome Resumido', 'Equipe', 'SKU','Produto', 'Fabricante', 'Unid. VDA', 'Qtde VDA',
       'Valor VDA', 'Total Venda']]

In [16]:
df['Aberto']

,Pedido,ID Cliente,Razão Social,Nome Fantasia,ID Vendedor,Nome Resumido,Equipe,SKU,Produto,Fabricante,Unid. VDA,Qtde VDA,Valor VDA,Total Venda
0,443618,10075,GRUPO FARTURA DE HORTIFRUT LTDA,REDE OBA PUJOL,JOSENUNE,JOSE SILVA,EQUIPE 4 (VAR),9447,BOLO CENOURA 330G PN10,BIMBO,CX,2.0,115.00,230.00
1,443619,23732,GRUPO FARTURA DE HORTIFRUT S.A,OBA ALPHAVILLE,JOSENUNE,JOSE SILVA,EQUIPE 4 (VAR),9447,BOLO CENOURA 330G PN10,BIMBO,CX,1.0,115.00,115.00
2,443634,25343,SUPERMERCADOS IRMÃOS LOPES SA,LOPES TRANQUILIDADE,JOSENUNE,JOSE SILVA,EQUIPE 4 (VAR),9447,BOLO CENOURA 330G PN10,BIMBO,CX,1.0,124.80,124.80
3,443752,20210,GRUPO FARTURA DE HORTIFRUT LTDA,REDE OBA TOLLE,JOSENUNE,JOSE SILVA,EQUIPE 4 (VAR),9447,BOLO CENOURA 330G PN10,BIMBO,CX,4.0,115.00,460.00
4,443759,8002,GRUPO FARTURA DE HORTIFRUT LTDA,OBA MACUCO,RODRIGOA,RODRIGO ANDRADE,EQUIPE 4 (VAR),9447,BOLO CENOURA 330G PN10,BIMBO,CX,1.0,115.00,115.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,443793,23877,VARANDA FRUTAS E MERCEARIA LTDA,VARANDA FRUTAS,RODRIGOA,RODRIGO ANDRADE,EQUIPE 4 (VAR),1078,POLPINHA SUCO DE TANGERINA (100 GR),DE MARCHI,KG,1.0,14.98,14.98
373,443767,14117,CRESCO COMERCIO DE ALIMENTOS LTDA,ST MARCHE IPIRANGA,DOUGLASM,DOUGLAS TEDESCHI,EQUIPE 4 (VAR),1078,POLPINHA SUCO DE TANGERINA (100 GR),DE MARCHI,CX,1.0,79.20,79.20
374,443799,8493,VIRTUS COMERCIO DE ALIMENTOS LTDA.,ST MARCHE SANTO AMAR,DOUGLASM,DOUGLAS TEDESCHI,EQUIPE 4 (VAR),1078,POLPINHA SUCO DE TANGERINA (100 GR),DE MARCHI,CX,1.0,79.20,79.20
375,443796,28868,LA VILA PRA VOCE MERCADO LTDA,LA VILA,RMORAES,RICARDO MORAES,EQUIPE 3 (CLT),2682,"CONGELADO PETISQUETE MANDIOCA (1,020KG) DE MARCHI",FRAGOLE,UN,5.0,20.57,102.85
